# Water retention curve - van Genuchten model

Water retention curve is the relationship between the water content, $θ$ (or effective Sauration $S_e$), and the soil water potential, $ψ_m$. This curve is characteristic for different types of soil, and is also called the soil moisture characteristic.

It is used to predict the soil water storage, water supply to the plants or available water (field capacity - wilting point). 

The water retention curve can partially obtained in the lab: basically this is done by applying a given suction to a saturated soil sample and then we measure the volume of water that the soil loses. In this way we estimate the content water content for a given suction pressure. However, in this way we only obtain a limited number of measurements or points of the curve. To obtain the whole curve we need a model that best fit the points.

The water retention curve can be obtained by several models, one of them known as the Van Genuchten model:

$$
S_{e} = \left[\frac{1}{1 + (-\alpha\psi_{m})^n}\right]^m\
$$

where $\alpha$ is a parameter of scale and $m$ and $n$ are parameters of shape. In general, $m = 1 - 1/n$ 

## Excercise 1

In the lab we have measured the effective saturation $S_e$ of a soil sample for the following suction pressures ($ψ_m$): -20, -60, -300, -800, -5000, -10000 cm and we have obtained these values for $S_e$: 0.951, 0.914, 0.432, 0.172, 0.042, 0.023

<left><img src="pressure_chamber.jpg" width="400px">

**Playing with the parameters $\alpha$ and $n$ try to find the best fit between the observed points and the modelled curve**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact # to create interactive elements and figures

Ψ_m_obs = np.array([20,60,300,800,5000,10000])
S_e_obs = np.array([0.951,0.914,0.432,0.172,0.042,0.023]) 

Ψ_m_sim = np.arange(1, 100000+1,1)

def vonGenuchten(Ψ_m, α, n, *args):
    
    m = 1 - 1/n
    
    if args:
        θ_r = args[0]
        θ_s = args[1]
        θ = θ_r + (θ_s - θ_r)*(1 / (1 + (-α * -Ψ_m)**n))**m
        result = θ
    else:
        S_e = (1 / (1 + (-α * -Ψ_m)**n))**m
        result = S_e
    
    return result

@interact(α = (0,0.05,0.01), n = (1,4,0.1))
def interactive_vonGenuchten_1(α = 0.004, n = 1.2):
    
    S_e_sim = vonGenuchten(Ψ_m_sim, α, n)
    
    MSE = np.square(np.subtract(S_e_obs,vonGenuchten(Ψ_m_obs, α, n))).mean() 
    RMSE = np.sqrt(MSE)

    ### Plot of the data and best fit line
    plt.scatter(Ψ_m_obs, S_e_obs, label = 'measured')
    plt.plot(Ψ_m_sim, S_e_sim, label = 'simulated')
    plt.ylabel('Volumetric water content [vol/vol]')
    plt.xlabel('Matric potential [|cm|]'); plt.xscale("log")
    plt.title('Goodness of fit (RMSE) = %.3f' %RMSE) ;plt.legend()
    plt.show()


interactive(children=(FloatSlider(value=0.004, description='α', max=0.05, step=0.01), FloatSlider(value=1.2, d…

## Excercise 2
Now we have modelled the water retention curve for three different soil textures, sand, loam and clay, using some typical values for the parameters of the van Genuchten model. Please notice that now we represent the curve in two different ways, one representing $S_e$ in the y axis and another one using the soil water content ($\theta$). This second way comes from writing down the equation as follows:

$$
S_{e} = \frac{\theta - \theta_r}{\theta_s - \theta_r} = \left[\frac{1}{1 + (-\alpha\psi_{m})^n}\right]^m\
$$


$$
\theta = \theta_r + (\theta_s - \theta_r) \left[\frac{1}{1 + (-\alpha\psi_{m})^n}\right]^m\
$$

where $\theta_r$ is the residual water content (water attached tightly to soil particles and that the plant cannot use) and $\theta_s$ is the saturation water content (when the all the soil pores are full of water).

**- Which soil texture fits better to the observed values?**

**- Which soil texture has more available water for plants?**

**- and a higher porosity?**

In [2]:
@interact(soil_type = ['sand', 'loam', 'clay'])
def interactive_vonGenuchten_2(soil_type = 'sand'):
    
    if soil_type == 'sand':
        α = 0.0124; n = 6.66; θ_r = 0.01; θ_s = 0.26
    elif soil_type == 'loam':
        α = 0.0081; n = 2.1632; θ_r = 0.05; θ_s = 0.43
    elif soil_type == 'clay':
        α = 0.0066; n = 1.8601; θ_r = 0.20; θ_s = 0.47 
    
    θ_obs = S_e_obs * (θ_s - θ_r) + θ_r
    
    θ_sim = vonGenuchten(Ψ_m_sim, α, n, θ_r, θ_s)
    S_e_sim = (θ_sim - θ_r)/(θ_s - θ_r)
    
    field_capacity = vonGenuchten(100, α, n, θ_r, θ_s); wilting_point = vonGenuchten(15000, α, n, θ_r, θ_s)
    field_capacity_S_e = (field_capacity - θ_r)/(θ_s - θ_r); wilting_point_S_e = (wilting_point - θ_r)/(θ_s - θ_r)
    
    #R2 = np.corrcoef(S_e_obs,vonGenuchten(Ψ_m_obs, α, n))

    plt.figure(figsize=(15,4)) # to define the plot size
    ### Plot of Effective saturation (Se) vs Matric potential (Ψ)
    plt.subplot(1,2,1)
    plt.scatter(Ψ_m_obs, S_e_obs, label = 'measured')
    plt.plot(Ψ_m_sim, S_e_sim, label = 'simulated')
    plt.vlines(100,0,field_capacity_S_e,linestyle= '--',color = 'g', label = 'field capacity')
    plt.vlines(15000,0,wilting_point_S_e,linestyle= '--',color = 'r', label = 'wilting point')
    plt.hlines(field_capacity_S_e,0,100,linestyle= '--',color = 'g')
    plt.hlines(wilting_point_S_e,0,15000,linestyle= '--',color = 'r')
    plt.ylabel('Se [vol/vol]')
    plt.xlabel('Ψm [|cm|]'); plt.xscale("log")
    plt.title('Effective saturation (Se) vs Matric potential (Ψ)');plt.legend()
    
    ### Plot of Soil water content (θ) vs Matric potential (Ψ)
    plt.subplot(1,2,2)
    plt.scatter(Ψ_m_obs, θ_obs, label = 'measured')
    plt.plot(Ψ_m_sim, θ_sim, label = 'simulated')
    plt.vlines(100,0,field_capacity,linestyle= '--',color = 'g', label = 'field capacity')
    plt.vlines(15000,0,wilting_point,linestyle= '--',color = 'r', label = 'wilting point')
    plt.hlines(field_capacity,0,100,linestyle= '--',color = 'g')
    plt.hlines(wilting_point,0,15000,linestyle= '--',color = 'r')
    plt.ylabel('θ [vol/vol]'); plt.ylim([0,1])
    plt.xlabel('Ψm [|cm|]'); plt.xscale("log")
    plt.title('Soil water content (θ) vs Matric potential (Ψ)');plt.legend()
    plt.show()

interactive(children=(Dropdown(description='soil_type', options=('sand', 'loam', 'clay'), value='sand'), Outpu…

## Excercise 3
Using the equation of the model to calculate $\theta$:

$$
\theta = \theta_r + (\theta_s - \theta_r) \left[\frac{1}{1 + (-\alpha\psi_{m})^n}\right]^m\
$$

**Try to find the combination of parameters that for you would be an ideal soil for agriculture.**

In [4]:
@interact(α = (0,0.1,0.01), n = (1,4,0.1), θ_r = (0,0.25,0.01), θ_s = (0.26,0.6,0.01))
def interactive_vonGenuchten_1(α = 0.004, n = 1.2, θ_r = 0.05, θ_s = 0.3):

    θ_sim = vonGenuchten(Ψ_m_sim, α, n, θ_r, θ_s)
    field_capacity = vonGenuchten(100, α, n, θ_r, θ_s); wilting_point = vonGenuchten(15000, α, n, θ_r, θ_s)
    available_water = field_capacity - wilting_point
    
    ### Plot of Soil water content (θ) vs Matric potential (Ψ)
    plt.plot(Ψ_m_sim, θ_sim)
    plt.vlines(100,0,field_capacity,linestyle= '--',color = 'g', label = 'field capacity')
    plt.vlines(15000,0,wilting_point,linestyle= '--',color = 'r', label = 'wilting point')
    plt.hlines(field_capacity,0,100,linestyle= '--',color = 'g')
    plt.hlines(wilting_point,0,15000,linestyle= '--',color = 'r')
    plt.ylabel('θ [vol/vol]'); plt.ylim([0,1])
    plt.xlabel('Ψm [|cm|]'); plt.xscale("log")
    plt.title('Available water (field capacity - wilting point) = %.3f' %available_water);plt.legend()
    plt.show()

interactive(children=(FloatSlider(value=0.004, description='α', max=0.1, step=0.01), FloatSlider(value=1.2, de…

**- what is the maximum value of available water that you can reach by playing with the values of the parameters?**

**- what combination of parameters represent the worst soil for agriculture?**